In [7]:
search_for = 120
start_from = 133819999
threads = 5
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004331350326538086
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 875
all cases: 1116151311
type: [1, 1, 1, 1, 120] 120
cases of this type: 207360000
50000 0.20013179670340087
100000 0.19269899347809288
150000 0.18645680396678446
200000 0.18378485865658656
250000 0.19287854522815545
300000 0.19534259401551485
350000 0.17807602877403628
400000 0.12051847730840445
450000 0.19064944377316234
500000 0.19838274478796855
550000 0.18899362309363155
600000 0.1886009763824145
650000 0.19248582693010569
700000 0.18058948820461035
750000 0.18863471268081666
800000 0.18892172396273085
850000 0.17968031890780395
900000 0.19252245705832244
950000 0.18380905566691821
1000000 0.17774777738666267
1050000 0.20126849472171468
1100000 0.2023479738017652
1150000 0.20559983453252978
1200000 0.18823961934156416
1250000 0.2062104863149974
1300000 0.18827294303295083
1350000 0.18933503958960773
1400000 0.1968554315105306
1450000 0.

14350000 0.19326679920584625
14400000 0.17995047027630406
14450000 0.1950587355313963
14500000 0.19099041726081636
14550000 0.17018486249246198
14600000 0.22989051356907822
14650000 0.1860417765655173
14700000 0.19954589275960924
14750000 0.18730284680166512
14800000 0.19511347061561213
14850000 0.19485954672230882
14900000 0.1863554649760167
14950000 0.18624636093812388
15000000 0.19008841675032379
15050000 0.19476290420815678
15100000 0.18726228539987802
15150000 0.18417233303678038
15200000 0.1853247461183151
15250000 0.16528071297090052
15300000 0.18546830727518399
15350000 0.13655788154645362
15400000 0.185686197168333
15450000 0.20077318382146758
15500000 0.188369294329483
15550000 0.1493689901665608
15600000 0.19410730854514838
15650000 0.18887129805827274
15700000 0.19008418588664663
15750000 0.19481544975382883
15800000 0.22312811117029585
15850000 0.19239886280121404
15900000 0.18606236991525094
15950000 0.19440843806699382
16000000 0.20246522069196038
16050000 0.133727256080

28850000 0.1989007121451537
28900000 0.1895279302007172
28950000 0.20711518467078605
29000000 0.1949997163255983
29050000 0.18593510679154793
29100000 0.17643793503261804
29150000 0.17226452146093846
29200000 0.18476487470451197
29250000 0.18641341561057173
29300000 0.12727670948787795
29350000 0.17667086667182574
29400000 0.1922433877494375
29450000 0.19014455197176802
29500000 0.13662471028705836
29550000 0.18114322344614264
29600000 0.20438642559007142
29650000 0.19160624483936495
29700000 0.18338041439591646
29750000 0.18213876027334266
29800000 0.18539673529000283
29850000 0.18536230284755228
29900000 0.16634301717896327
29950000 0.18322168052971363
30000000 0.1877162497034073
30050000 0.20419388358240656
30100000 0.19520807650711802
30150000 0.18902665545834302
30200000 0.18663453327930238
30250000 0.19794270518067147
30300000 0.2040476419434031
30350000 0.19446417817215653
30400000 0.1845163112948113
30450000 0.18941448553186654
30500000 0.18717830518567033
30550000 0.1897335150

43350000 0.18707155200262862
43400000 0.18018846942389277
43450000 0.17979075973738298
43500000 0.17995429356886147
43550000 0.18026840679850312
43600000 0.18187527721213367
43650000 0.18166081461384295
43700000 0.18640254064671463
43750000 0.1849918693146242
43800000 0.1862260770601829
43850000 0.19067860087164376
43900000 0.18530685345692768
43950000 0.1867084529062748
44000000 0.18771796894674173
44050000 0.21494860005661778
44100000 0.1855832656752189
44150000 0.1857379005890409
44200000 0.18591523788020345
44250000 0.18423122050412497
44300000 0.1833239452514344
44350000 0.18566349778425828
44400000 0.19555360753734113
44450000 0.18009301506845027
44500000 0.17943024492381418
44550000 0.17494938478195268
44600000 0.17432807521498866
44650000 0.17476274825812313
44700000 0.17462828545560838
44750000 0.17891865715311234
44800000 0.17864920112255284
44850000 0.17845639490457776
44900000 0.17894327022344803
44950000 0.17840426556602848
45000000 0.1860758537570119
45050000 0.1897203841

57900000 0.11820011675541402
57950000 0.16896606794958513
58000000 0.18145590044382678
58050000 0.18161410541790723
58100000 0.18238472364326452
58150000 0.18450619195245505
58200000 0.1497590734665672
58250000 0.1793504031501002
58300000 0.17714068133374455
58350000 0.20287206838519575
58400000 0.20945017866195043
58450000 0.20822983484341304
58500000 0.20746490107632082
58550000 0.2109964252367695
58600000 0.2044518247555574
58650000 0.20411579283456405
58700000 0.17120219759787586
58750000 0.1997518968265997
58800000 0.17803311169719693
58850000 0.176918340548714
58900000 0.18201995975768434
58950000 0.182092569773074
59000000 0.18198874094666243
59050000 0.18149427060874568
59100000 0.18242656325689158
59150000 0.18230733038566513
59200000 0.18613325834631123
59250000 0.18275345503728388
59300000 0.18096839975270562
59350000 0.1775065716742277
59400000 0.17712024966073037
59450000 0.17782709103671845
59500000 0.16840506791548598
59550000 0.17658708309868573
59600000 0.1203669154102

...

best so far: 0
type: [2, 3, 10, 2, 1] 120
cases of this type: 2400
best so far: 0
type: [2, 3, 20, 1, 1] 120
cases of this type: 1200
best so far: 0
type: [2, 4, 1, 1, 15] 120
cases of this type: 202500
1101750000 0.16547402896147964
1101800000 0.233931902134245
1101850000 0.12790306631137133
1101900000 0.25368267938560246
best so far: 0
type: [2, 4, 1, 3, 5] 120
cases of this type: 67500
1101950000 0.3314121379283269
1102000000 0.23914351455409663
best so far: 0
type: [2, 4, 1, 5, 3] 120
cases of this type: 40500
best so far: 0
type: [2, 4, 1, 15, 1] 120
cases of this type: 13500
1102050000 0.18098648780232665
best so far: 0
type: [2, 4, 3, 1, 5] 120
cases of this type: 22500
best so far: 0
type: [2, 4, 3, 5, 1] 120
cases of this type: 4500
best so far: 0
type: [2, 4, 5, 1, 3] 120
cases of this type: 8100
best so far: 0
type: [2, 4, 5, 3, 1] 120
cases of this type: 2700
best so far: 0
type: [2, 4, 15, 1, 1] 120
cases of this type: 900
best so far: 0
type: [2, 5, 1, 1, 12] 120
cases of

1107800000 0.11404424445540508
1107850000 0.11234805338517295
1107900000 0.1300760220105211
best so far: 0
type: [3, 1, 1, 10, 4] 120
cases of this type: 256000
1107950000 0.1313336525334888
1108000000 0.1312098709625575
1108050000 0.12875851138264735
1108100000 0.12735952859402366
1108150000 0.21597015032380287
best so far: 0
type: [3, 1, 1, 20, 2] 120
cases of this type: 128000
1108200000 0.15991990987837315
1108250000 0.10217183051789601
1108300000 0.10122856395997473
best so far: 0
type: [3, 1, 1, 40, 1] 120
cases of this type: 64000
1108350000 0.09979660622978606
best so far: 0
type: [3, 1, 2, 1, 20] 120
cases of this type: 640000
1108400000 0.2728440615244402
1108450000 0.22661530810039043
1108500000 0.19757844395473004
1108550000 0.1856465105983774
1108600000 0.1707806369396011
1108650000 0.10361733959388733
1108700000 0.06095692319925626
1108750000 0.06069352746094068
1108800000 0.10329449004401764
1108850000 0.16002699228607417
1108900000 0.15063586450789504
1108950000 0.19047

1112650000 0.06029784992326895
1112700000 0.047754998533105844
1112750000 0.043892581485075424
best so far: 0
type: [4, 1, 1, 5, 6] 120
cases of this type: 162000
1112800000 0.04193368201710383
1112850000 0.03487076874112288
1112900000 0.03547152224580315
1112950000 0.03489831627489991
best so far: 0
type: [4, 1, 1, 6, 5] 120
cases of this type: 135000
1113000000 0.03316617113519907
1113050000 0.033569562303772234
best so far: 0
type: [4, 1, 1, 10, 3] 120
cases of this type: 81000
1113100000 0.03369663440821569
1113150000 0.03987798154927492
best so far: 0
type: [4, 1, 1, 15, 2] 120
cases of this type: 54000
1113200000 0.03017734061281416
best so far: 0
type: [4, 1, 1, 30, 1] 120
cases of this type: 27000
1113250000 0.024793074486753677
best so far: 0
type: [4, 1, 2, 1, 15] 120
cases of this type: 202500
1113300000 0.04526972607877652
1113350000 0.03380152027586831
1113400000 0.04066143800274531
1113450000 0.04202341604054769
best so far: 0
type: [4, 1, 2, 3, 5] 120
cases of this type:

best so far: 0
type: [5, 2, 2, 1, 6] 120
cases of this type: 10368
best so far: 0
type: [5, 2, 2, 2, 3] 120
cases of this type: 5184
best so far: 0
type: [5, 2, 2, 3, 2] 120
cases of this type: 3456
best so far: 0
type: [5, 2, 2, 6, 1] 120
cases of this type: 1728
best so far: 0
type: [5, 2, 3, 1, 4] 120
cases of this type: 4608
1115400000 0.007129328286957741
best so far: 0
type: [5, 2, 3, 2, 2] 120
cases of this type: 2304
best so far: 0
type: [5, 2, 3, 4, 1] 120
cases of this type: 1152
best so far: 0
type: [5, 2, 4, 1, 3] 120
cases of this type: 2592
best so far: 0
type: [5, 2, 4, 3, 1] 120
cases of this type: 864
best so far: 0
type: [5, 2, 6, 1, 2] 120
cases of this type: 1152
best so far: 0
type: [5, 2, 6, 2, 1] 120
cases of this type: 576
best so far: 0
type: [5, 2, 12, 1, 1] 120
cases of this type: 288
best so far: 0
type: [5, 3, 1, 1, 8] 120
cases of this type: 12288
best so far: 0
type: [5, 3, 1, 2, 4] 120
cases of this type: 6144
best so far: 0
type: [5, 3, 1, 4, 2] 120
cas

best so far: 0
type: [10, 2, 1, 3, 2] 120
cases of this type: 864
best so far: 0
type: [10, 2, 1, 6, 1] 120
cases of this type: 432
best so far: 0
type: [10, 2, 2, 1, 3] 120
cases of this type: 648
best so far: 0
type: [10, 2, 2, 3, 1] 120
cases of this type: 216
best so far: 0
type: [10, 2, 3, 1, 2] 120
cases of this type: 288
best so far: 0
type: [10, 2, 3, 2, 1] 120
cases of this type: 144
best so far: 0
type: [10, 2, 6, 1, 1] 120
cases of this type: 72
best so far: 0
type: [10, 3, 1, 1, 4] 120
cases of this type: 768
best so far: 0
type: [10, 3, 1, 2, 2] 120
cases of this type: 384
best so far: 0
type: [10, 3, 1, 4, 1] 120
cases of this type: 192
best so far: 0
type: [10, 3, 2, 1, 2] 120
cases of this type: 192
best so far: 0
type: [10, 3, 2, 2, 1] 120
cases of this type: 96
best so far: 0
type: [10, 3, 4, 1, 1] 120
cases of this type: 48
best so far: 0
type: [10, 4, 1, 1, 3] 120
cases of this type: 324
best so far: 0
type: [10, 4, 1, 3, 1] 120
cases of this type: 108
best so far: 